In [3]:
!pip install pandas numpy scikit-learn python-whois



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer


In [20]:

# Load the dataset
dataset_path = 'cybersecurity_large_synthesized_data.csv'  # Update this path if necessary
data = pd.read_csv(dataset_path)

# Display the first few rows of the dataset
data.head()



attack_type  target_system  outcome                   timestamp  \
0          Phishing  Cloud Service  Failure  2024-04-03 11:13:15.083419   
1              DDoS   Email Server  Success  2024-02-03 20:51:56.083463   
2  Zero-Day Exploit  Cloud Service  Success  2024-07-19 18:40:05.083472   
3     SQL Injection   Email Server  Failure  2023-12-08 00:54:17.083478   
4  Zero-Day Exploit     IoT Device  Failure  2024-07-23 05:38:20.083483   

       attacker_ip        target_ip  data_compromised_GB  attack_duration_min  \
0      21.166.79.4      23.20.50.78                15.31                  300   
1  187.180.150.169    34.160.58.218                65.05                  242   
2   57.161.159.140  213.142.125.206                48.99                  120   
3   207.108.16.104     4.104.193.49                16.29                  286   
4     98.122.15.31     57.66.53.194                87.66                    4   

  security_tools_used      user_role   location  attack_severity    industry  \
0            Firewall       Employee  Australia                2      Energy   
1  Endpoint Detection          Admin     Brazil               10      Retail   
2            Firewall  External User    Germany                7     Finance   
3                 MFA       Employee     Russia                3  Healthcare   
4            Firewall     Contractor         UK                6     Finance   

   response_time_min  mitigation_method  
0                164        Containment  
1                 64  Reset Credentials  
2                 87         Quarantine  
3                 23              Patch  
4                129        Containment

In [22]:
# Display column names
print(data.columns)


Index(['attack_type', 'target_system', 'outcome', 'timestamp', 'attacker_ip',
       'target_ip', 'data_compromised_GB', 'attack_duration_min',
       'security_tools_used', 'user_role', 'location', 'attack_severity',
       'industry', 'response_time_min', 'mitigation_method'],
      dtype='object')


In [23]:
label_mapping = {'benign': 0, 'phishing': 1, 'defacement': 2}
data['Label'] = data['Label'].map(label_mapping)  # Use correct column name

# Check for missing labels or anomalies
print(data['Label'].value_counts())


KeyError: 'Label'

In [8]:
def extract_features(api_row):
    features = {}

    # Endpoint length
    features['endpoint_length'] = len(api_row['endpoint'])

    # Count of query parameters
    features['num_parameters'] = len(api_row['parameters'].split('&')) if pd.notna(api_row['parameters']) else 0

    # Count special characters in the endpoint
    features['num_special_chars'] = len(re.findall(r'[^a-zA-Z0-9/:._-]', api_row['endpoint']))

    # HTTP method type (convert to numeric encoding)
    method_mapping = {'GET': 1, 'POST': 2, 'PUT': 3, 'DELETE': 4, 'OTHER': 0}
    features['method_type'] = method_mapping.get(api_row['method'], 0)

    # Check if endpoint starts with "/api/"
    features['starts_with_api'] = 1 if api_row['endpoint'].startswith('/api/') else 0

    # Count number of words in endpoint
    features['num_words_in_endpoint'] = len(api_row['endpoint'].strip('/').split('/'))

    # Ratio of alphabetic to total characters in the endpoint
    alpha_chars = len(re.findall(r'[a-zA-Z]', api_row['endpoint']))
    features['alpha_ratio'] = alpha_chars / len(api_row['endpoint']) if len(api_row['endpoint']) > 0 else 0

    # Presence of keywords in the endpoint
    keywords = ['user', 'login', 'token', 'data', 'auth']
    features['contains_keyword'] = 1 if any(keyword in api_row['endpoint'].lower() for keyword in keywords) else 0

    # Total length of parameter keys and values
    features['param_length'] = sum(len(param) for param in api_row['parameters'].split('&')) if pd.notna(api_row['parameters']) else 0

    return features


In [9]:
# Apply feature extraction
features_df = pd.DataFrame([extract_features(row) for _, row in data.iterrows()])

# Add labels from the dataset
features_df['label'] = data['label']

# Display the features
features_df.head()


NameError: name 'data' is not defined

In [10]:
# Split the data into features and target
X = features_df.drop('label', axis=1)
y = features_df['label']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


NameError: name 'features_df' is not defined

In [11]:
# Train the RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


NameError: name 'X_train' is not defined

In [12]:
# Predict and evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_mapping.keys()))


NameError: name 'X_test' is not defined

In [13]:
def predict_api_from_input():
    print("\nEnter API details for prediction:")
    method = input("HTTP Method (GET/POST/PUT/DELETE): ").strip()
    endpoint = input("API Endpoint (e.g., /api/v1/resource): ").strip()
    parameters = input("Query Parameters (e.g., id=123&key=abc): ").strip()

    # Construct a test API dictionary
    test_api = {
        'method': method,
        'endpoint': endpoint,
        'parameters': parameters
    }

    # Extract features and predict
    features = extract_features(test_api)
    features_df = pd.DataFrame([features])
    prediction = model.predict(features_df)
    predicted_label = prediction[0]

    print(f"\nPrediction: {predicted_label} (0 = benign, 1 = phishing, 2 = defacement)")  # Adjust labels as per your dataset


In [14]:
predict_api_from_input()



Enter API details for prediction:


HTTP Method (GET/POST/PUT/DELETE):  exit


KeyboardInterrupt: Interrupted by user

In [ ]:
import joblib

# Save the model
joblib.dump(model, 'api_detection_model.pkl')

# To load the model later
# model = joblib.load('api_detection_model.pkl')
